In [1]:
from atomate2.lobster.schemas import LobsterTaskDocument
from pymatgen.core import Structure
import os
from lobsterpy.cohp.analyze import Analysis
from lobsterpy.cohp.describe import Description
import warnings
import json
import gzip
import multiprocessing as mp
from multiprocessing.pool import ThreadPool
from tqdm import tqdm
import gc
warnings.filterwarnings('ignore')

In [ ]:
parent=os.getcwd()
os.chdir(parent)
os.chdir('Results/')

In [ ]:
mpids= [f for f in os.listdir() if not f.startswith('t') and not f.startswith('.') and not f.startswith('__')
            and os.path.isdir(f)]
mats= list(set([ids.split('_')[0] for ids in mpids]))
mats.sort()
os.chdir(parent)

In [ ]:
def create_jsons(mpid, json_save_dir='/hpc-user/AG-JGeorge/anaik/Phonon_dataset_LSO/Json_data/'):
    parent=os.getcwd()
    #print(parent)
    os.chdir('Results')
    os.chdir(mpid)
    path= os.getcwd()
    #print(path)
    output = LobsterTaskDocument.from_directory(dir_name=path, additional_fields=['DOSCAR.LSO.lobster'])
    #json_out = output.copy(deep=True)
    output.structure = output.structure.as_dict()
    output.cobi_data = output.cobi_data.as_dict()
    output.cohp_data = output.cohp_data.as_dict()
    output.coop_data = output.coop_data.as_dict()
    output.dos = output.dos.as_dict()
    output.lso_dos = output.lso_dos.as_dict()
    os.chdir(parent)
    data = output.dict(exclude={'dir_name','last_updated'})
    del output
    gc.collect()
    with gzip.open(os.path.join(json_save_dir, "{}.json.gz".format(mpid)), 'wt', encoding='UTF-8') as f:
        json.dump(data, f)
    return mpid+' Done'

In [ ]:
items=mats
with mp.Pool(processes=3,maxtasksperchild=1) as pool:
    results = tqdm(
        pool.imap(create_jsons, items, chunksize=1),
        total=len(items),
    )
    row=[]
    for result in results:
        row.append(result)